In [10]:
double_dict = {
    "paras" : {
        "h" : 3,
        "w" : 2
    }
}

def f(*args):
    for arg in args:
        print(arg)

f(1)
f("2nd",1,2)

def g(**kwargs):
    for k in kwargs:
        print(k, kwargs[k])

test_dict = {
    "title" : "Harry Poter",
    "price" : 0.3
}

g(**test_dict)
g(**double_dict["paras"])


1
2nd
1
2
title Harry Poter
price 0.3
h 3
w 2


In [16]:
import torch
import torch.nn as nn
model = torch.nn.Conv2d(1, 1, 3, 1, padding=0)
inputs = torch.Tensor([[[1,2,3],[4,5,6],[7,8,9]]])
print(inputs)
print(model(inputs))

tensor([[[1., 2., 3.],
         [4., 5., 6.],
         [7., 8., 9.]]])
tensor([[[-3.2907]]], grad_fn=<SqueezeBackward1>)


In [7]:
import torch

t = torch.Tensor([[[1,2,3], [4,5,6]]])
print(t.dtype, t.shape, t)

t_ = t.flatten(1)
print(t_.dtype, t_.shape, t_)

torch.float32 torch.Size([1, 2, 3]) tensor([[[1., 2., 3.],
         [4., 5., 6.]]])
torch.float32 torch.Size([1, 6]) tensor([[1., 2., 3., 4., 5., 6.]])


In [13]:
class A:
    def __init__(self):
        print("class A constructed")

a = A()
a.device = "hey"

print(A, a)
print(a.device)

c = [1, 2]
c.device = "ho"

print(c)

class A constructed
<class '__main__.A'> <__main__.A object at 0x12e6c2f80>
hey


AttributeError: 'list' object has no attribute 'device'

In [27]:
import torch

a = torch.randn((10, 3))
print(a)

b = (a.argmax(dim=-1) >= 1)
print(b)

c = torch.zeros((10, 3))
print(c)




tensor([[ 1.7390, -0.3980, -0.1157],
        [ 0.1020, -0.6843, -0.8908],
        [-0.4062, -0.4279, -1.7917],
        [-0.8333,  0.7010,  1.1794],
        [-0.7411,  1.7690,  1.4698],
        [-0.5558,  1.4585, -0.5355],
        [ 0.4470,  0.2723, -0.8688],
        [-0.5657, -0.9404,  1.1317],
        [ 0.5942,  1.1422,  1.7809],
        [ 0.6076,  0.0473,  1.7281]])
tensor([False, False, False,  True,  True,  True, False,  True,  True,  True])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [51]:
import torch

a = torch.Tensor([[5, 2, 3, 4], [5, 6, 7, 8]])
print(a)
b = a.argmax(dim=-1).tolist()
c = a.max(dim=-1)
d = {"value" : c[0], "index" : c[1]}

e = d["value"].numpy()

print(e)
e += 1
e


tensor([[5., 2., 3., 4.],
        [5., 6., 7., 8.]])
[5. 8.]


array([6., 9.], dtype=float32)

In [21]:
a = ("aaa", 1)
print(type(a))
a[1]

<class 'tuple'>


1

In [3]:
import torch
from torch.utils.data import ConcatDataset, Dataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder
from torchvision.transforms import v2
from PIL import Image
from tqdm.auto import tqdm


class TestDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    
xs = torch.Tensor([1, 2, 3, 4])
ys = torch.Tensor([5, 6, 7, 8])

t = TestDataset(xs, ys)

t_loader = DataLoader(t)

for i, (x, y) in enumerate(t_loader):
    t.y[i] = 9


tfm =v2.Compose([
    v2.Resize((128, 128)),
    v2.ToTensor()
])
train_unlb_dataset = DatasetFolder("./food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=tfm)
train_unlb_loader = DataLoader(train_unlb_dataset)

class Custom_DatasetFolder(DatasetFolder):

    def __init__(self):

        self.data = []
        self.label = []

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]
    
    def add_sample(self, data, label):
        self.data.append(data)
        self.label.append(label)

class Custom_Dataset(Dataset):
    def __init__(self):
        self.data = []
        self.label = []

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.label[index]
    
    def datasetFolder_feed(self, datasetFolder):
        for i in range(len(datasetFolder)):
            self.data.append(datasetFolder.samples[i][0])
            self.label.append(datasetFolder.samples[i][1])
    
    def add_samples(self, sample):
        self.data.append(sample[0])
        self.label.append(sample[1])
    
    def set_label(self, index, label):
        if index in range(len(self.label)):
            self.label[index] = label


train_set = Custom_Dataset()
train_set.datasetFolder_feed(train_unlb_dataset)

subset = Subset(train_set, [0, 1])

pseudo_set = Custom_Dataset()
pseudo_set.add_samples(train_set.__getitem__(2))

concatset = ConcatDataset([subset, pseudo_set])

c_loader = DataLoader(concatset, batch_size=3)

for batch in tqdm(c_loader):
    data, label = batch
    print(data)
    print(label)

c_datasetFolder = Custom_DatasetFolder()
c_datasetFolder.add_sample(train_unlb_dataset.samples[0][0], 999)

cf_loader = DataLoader(c_datasetFolder)

for batch in tqdm(cf_loader):
    data, label = batch
    print(data)
    print(label)




/Users/complexa/anaconda3/envs/complexa_ML21_3a/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

('./food-11/training/unlabeled/00/0000.jpg', './food-11/training/unlabeled/00/0001.jpg', './food-11/training/unlabeled/00/0002.jpg')
tensor([0, 0, 0])


  0%|          | 0/1 [00:00<?, ?it/s]

('./food-11/training/unlabeled/00/0000.jpg',)
tensor([999])


In [54]:
import torch

lst = [2, 4, 6]

a = [i for i in range(10) if i not in lst]
a

[0, 1, 3, 5, 7, 8, 9]

In [25]:
import torch
from torch.utils.data import ConcatDataset, Dataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder
from torchvision.transforms import v2
from PIL import Image
from tqdm.auto import tqdm



tfm =v2.Compose([
    v2.Resize((128, 128)),
    v2.ToTensor()
])

train_lb_dataset = DatasetFolder("./food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=tfm)


class ddd(DatasetFolder):
    def __init__(self, root, loader, extension, transform):
        super().__init__(root=root, loader=loader, extensions=extension, transform=transform)
    
    def __getitem__(self, index: int):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (sample, target) where target is class_index of the target class.
        """
        path, target = self.samples[index]
        sample = self.loader(path)
        if self.transform is not None:
            sample = self.transform(sample)

        return sample, self.targets[index]
    
    def set_label(self, index, label):
        self.targets[index] = label
    
a = ddd("./food-11/training/unlabeled", lambda x: Image.open(x), "jpg", tfm)
a.set_label(0, 999)

sub = Subset(a, [2, 5, 9, 11])

c = ConcatDataset([train_lb_dataset, sub])
dl = DataLoader(c, batch_size=128, shuffle=True, pin_memory=True)

'''
for i in tqdm(dl):
    print(i)
'''

print(sub[0])
print(a[2])

(tensor([[[0.8471, 0.8627, 0.8627,  ..., 0.0706, 0.0667, 0.0588],
         [0.8706, 0.8941, 0.8510,  ..., 0.0745, 0.0667, 0.0588],
         [0.9137, 0.9137, 0.8863,  ..., 0.0980, 0.0627, 0.0549],
         ...,
         [0.7882, 0.8157, 0.8235,  ..., 0.9529, 0.9647, 0.9608],
         [0.8039, 0.8196, 0.8157,  ..., 0.9608, 0.9765, 0.9804],
         [0.8118, 0.8118, 0.8000,  ..., 0.9647, 0.9804, 0.9843]],

        [[0.7333, 0.7451, 0.6980,  ..., 0.0902, 0.0824, 0.0824],
         [0.7529, 0.7529, 0.6588,  ..., 0.1020, 0.0863, 0.0863],
         [0.9255, 0.8784, 0.7373,  ..., 0.1373, 0.0902, 0.0902],
         ...,
         [0.8275, 0.8549, 0.8588,  ..., 0.9882, 1.0000, 0.9961],
         [0.8392, 0.8588, 0.8588,  ..., 0.9922, 1.0000, 1.0000],
         [0.8392, 0.8471, 0.8431,  ..., 0.9882, 0.9961, 1.0000]],

        [[0.7333, 0.7451, 0.6941,  ..., 0.0902, 0.0784, 0.0784],
         [0.7569, 0.7647, 0.6627,  ..., 0.1098, 0.0863, 0.0824],
         [0.9059, 0.8706, 0.7412,  ..., 0.1529, 0.0941, 0

/Users/complexa/anaconda3/envs/complexa_ML21_3a/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


In [17]:
import torch

probs = torch.randint(0, 10, (5, 8))

max_prob_idx = probs.max(dim=-1)
max_prob = max_prob_idx[0]
max_idx = max_prob_idx[1]

max_idx[0]

tensor(0)

In [29]:
a = list(range(10))

for i in range(len(a)):
    if a[i] % 2 != 0:
        if i in [3, 4, 5]:
            continue
        a[i] = 999

a

[0, 999, 2, 3, 4, 5, 6, 999, 8, 999]

In [5]:
a = ("str", 1)
print(a)
b = list(a)
print(b)

b[1] = 2

a = tuple(b)
print(a, type(a))

('str', 1)
['str', 1]
('str', 2) <class 'tuple'>
